In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#We will do credit modeling
#Given some dataset we have to predict if a loan will be paid off or not.
#So we have two type of dataset
#1)gives the description about the columns
#2)the dataset about the borrower

In [3]:
#Read the dataset
credit = pd.read_csv("loans_2007.csv")

C:\Users\hp\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42538 entries, 0 to 42537
Data columns (total 52 columns):
id                            42538 non-null object
member_id                     42535 non-null float64
loan_amnt                     42535 non-null float64
funded_amnt                   42535 non-null float64
funded_amnt_inv               42535 non-null float64
term                          42535 non-null object
int_rate                      42535 non-null object
installment                   42535 non-null float64
grade                         42535 non-null object
sub_grade                     42535 non-null object
emp_title                     39911 non-null object
emp_length                    42535 non-null object
home_ownership                42535 non-null object
annual_inc                    42531 non-null float64
verification_status           42535 non-null object
issue_d                       42535 non-null object
loan_status                   42535 non-null object
p

In [5]:
#So we have 52 columns now lets read about all the columns 

In [6]:
columns = credit.columns

In [7]:
columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt',
       'pub_rec_bankruptcies', 'tax_liens'],
      dtype='object')

In [8]:
#First we will take 18 columns each and see what we are going to find out

In [9]:
#'id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
 #      'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
  #     'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
   #    'issue_d', 'loan_status', 'pymnt_plan'

In [10]:
#so we have to look for 3 things like
#1)We should not have any column that should leak the data from future
#2)Redundant columns
#3)columns which need more processing so that it can be useful for algorithms

In [11]:
#In the first 18 columns id,member_id are not necessary because these are just random numbers
#So we will drop that column
#funded_amnt,funded_amnt_inv leaks data from the future 
#needs to be drop that also,otherwise our model will overfit
#depending upon the intrest rate grade and sub_grade are given some values,Means they are redundant
#emp_tile needs more processing to be useful
#issue_d is also leaks data from future
#so these are the columns which are need to be droped lets drop these


In [12]:
credit = credit.drop(['id', 'member_id','funded_amnt', 'funded_amnt_inv','grade', 'sub_grade', 'emp_title','issue_d'],axis=1)

In [13]:
#lets analyze next 18 columns
#'purpose', 'title', 'zip_code',
 #      'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
  #     'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
   #    'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
   #    'total_pymnt', 'total_pymnt_inv', 

In [14]:
#zipcode is redundant as addr_state is there
#'out_prncp', 'out_prncp_inv',
   #    'total_pymnt', 'total_pymnt_inv' leaks data from  future
#lets remove these columns

In [15]:
credit = credit.drop(['out_prncp', 'out_prncp_inv','total_pymnt', 'total_pymnt_inv','zip_code'],axis=1)

In [16]:
#lets analyze last 16 columns
# 'total_rec_prncp', 'total_rec_int',
  #     'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
   #    'last_pymnt_d', 'last_pymnt_amnt', 'last_credit_pull_d',
    #   'collections_12_mths_ex_med', 'policy_code', 'application_type',
     #  'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt',
      # 'pub_rec_bankruptcies', 'tax_liens'

In [17]:
#total_rec_prncp', 'total_rec_int',
  #     'total_rec_late_fee', 'recoveries', 'collection_recovery_fee','last_pymnt_d', 'last_pymnt_amnt'
    #leaks data from future

In [18]:
credit = credit.drop(['total_rec_prncp', 'total_rec_int','total_rec_late_fee', 'recoveries', 'collection_recovery_fee','last_pymnt_d', 'last_pymnt_amnt'],axis=1)

In [19]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42538 entries, 0 to 42537
Data columns (total 32 columns):
loan_amnt                     42535 non-null float64
term                          42535 non-null object
int_rate                      42535 non-null object
installment                   42535 non-null float64
emp_length                    42535 non-null object
home_ownership                42535 non-null object
annual_inc                    42531 non-null float64
verification_status           42535 non-null object
loan_status                   42535 non-null object
pymnt_plan                    42535 non-null object
purpose                       42535 non-null object
title                         42523 non-null object
addr_state                    42535 non-null object
dti                           42535 non-null float64
delinq_2yrs                   42506 non-null float64
earliest_cr_line              42506 non-null object
inq_last_6mths                42506 non-null float64
o

In [20]:
#Now we are finish with all columns
credit.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,Fully Paid,n,...,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,Charged Off,n,...,f,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,Fully Paid,n,...,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,Fully Paid,n,...,f,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,3000.0,60 months,12.69%,67.79,1 year,RENT,80000.0,Source Verified,Current,n,...,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [21]:
columns = credit.columns

In [22]:
columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'loan_status',
       'pymnt_plan', 'purpose', 'title', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'initial_list_status',
       'last_credit_pull_d', 'collections_12_mths_ex_med', 'policy_code',
       'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths',
       'delinq_amnt', 'pub_rec_bankruptcies', 'tax_liens'],
      dtype='object')

In [23]:
#Our loan_status is the predicted column

In [24]:
credit['loan_status'].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

In [25]:
#so here fully paid means loans have been fully paid
#so charged off means loans will not be paid to the investers

In [26]:
# so we need to remove other rows containing values except fully paid and charged off

In [27]:
#so actually we are removing 0.09% data from our dataset which is kind of correct, we are not going to
#loose that much of information

In [28]:
truth = (credit['loan_status']=='Fully Paid') | (credit['loan_status']=='Charged Off')

In [29]:
credit = credit[truth]

In [30]:
credit['loan_status'].value_counts()

Fully Paid     33136
Charged Off     5634
Name: loan_status, dtype: int64

In [31]:
#now we have only fully paid and charged off columns
#see the values of fullypaid and charged off we can say it is a class imbalance problem

In [32]:
#first we need to change the fully paid and charged off column to numerical values .
#fully paid =1 and charged off=0

In [33]:
loan_status_numerical = {"loan_status":{"Fully Paid":1,"Charged Off":0}}

In [34]:
credit = credit.replace(loan_status_numerical)

In [35]:
credit.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,1,n,...,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,0,n,...,f,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,1,n,...,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,1,n,...,f,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
5,5000.0,36 months,7.90%,156.46,3 years,RENT,36000.0,Source Verified,1,n,...,f,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [36]:
#Then here int_rate is object type lets convert it to float type removing the % sign

In [37]:
credit['int_rate'] = credit['int_rate'].str.rstrip('%').astype("float")

In [38]:
credit.head(2)

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65,162.87,10+ years,RENT,24000.0,Verified,1,n,...,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27,59.83,< 1 year,RENT,30000.0,Source Verified,0,n,...,f,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [39]:
#lets look at all the object columns since we have to convert it to the numerical values to be suitable 
#for machine learning algorithms to work

In [40]:
object_column = credit.select_dtypes(include=['object'])

In [41]:
object_column.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38770 entries, 0 to 39785
Data columns (total 13 columns):
term                   38770 non-null object
emp_length             38770 non-null object
home_ownership         38770 non-null object
verification_status    38770 non-null object
pymnt_plan             38770 non-null object
purpose                38770 non-null object
title                  38760 non-null object
addr_state             38770 non-null object
earliest_cr_line       38770 non-null object
revol_util             38720 non-null object
initial_list_status    38770 non-null object
last_credit_pull_d     38768 non-null object
application_type       38770 non-null object
dtypes: object(13)
memory usage: 4.1+ MB


In [42]:
#so 13 columns contains the object type
#lets first convert the emp_length

In [43]:
credit['emp_length'].value_counts()

10+ years    8547
< 1 year     4527
2 years      4308
3 years      4026
4 years      3362
5 years      3209
1 year       3183
6 years      2181
7 years      1718
8 years      1444
9 years      1229
n/a          1036
Name: emp_length, dtype: int64

In [44]:
length_emp = {"emp_length":{
    "< 1 year":0.0,
    "1 year":1.0,
    "2 years":2.0,
    "3 years":3.0,
    "4 years":4.0,
    "5 years":5.0,
    "6 years":6.0,
    "7 years":7.0,
    "8 years":8.0,
    "9 years":9.0,
    "10+ years":10.0,
    "n/a":0
}} 

In [45]:
credit = credit.replace(length_emp)

In [46]:
credit['emp_length']=credit['emp_length'].astype('int')

In [47]:
credit.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,...,initial_list_status,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,5000.0,36 months,10.65,162.87,10,RENT,24000.0,Verified,1,n,...,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,2500.0,60 months,15.27,59.83,0,RENT,30000.0,Source Verified,0,n,...,f,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,2400.0,36 months,15.96,84.33,10,RENT,12252.0,Not Verified,1,n,...,f,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,10000.0,36 months,13.49,339.31,10,RENT,49200.0,Source Verified,1,n,...,f,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
5,5000.0,36 months,7.90,156.46,3,RENT,36000.0,Source Verified,1,n,...,f,Jan-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [48]:
#then we need to find out the columns which have only one unique value.we have to remove them.because
#they donot change they are constant.

In [49]:
credit['title'].unique()

array(['Computer', 'bike', 'real estate business', ...,
       'Retiring credit card debt', 'MBA Loan Consolidation', 'JAL Loan'], dtype=object)

In [50]:
unique=[]
def uni(name):
    s = credit[name]
    x = s.dropna().unique()
    if len(x) ==1:
        unique.append(name)
    

In [51]:
columns = credit.columns

In [52]:
for i in columns:
    uni(i)

In [53]:
unique

['pymnt_plan',
 'initial_list_status',
 'collections_12_mths_ex_med',
 'policy_code',
 'application_type',
 'acc_now_delinq',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'tax_liens']

In [54]:
#lets drop these columns

In [55]:
credit.drop(unique,axis=1,inplace=True)

In [56]:
credit.head(2)

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,...,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_credit_pull_d,pub_rec_bankruptcies
0,5000.0,36 months,10.65,162.87,10,RENT,24000.0,Verified,1,credit_card,...,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,Jun-2016,0.0
1,2500.0,60 months,15.27,59.83,0,RENT,30000.0,Source Verified,0,car,...,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,Sep-2013,0.0


In [57]:
#now lets see how many null values are there in every column

In [58]:
credit.isnull().sum()

loan_amnt                 0
term                      0
int_rate                  0
installment               0
emp_length                0
home_ownership            0
annual_inc                0
verification_status       0
loan_status               0
purpose                   0
title                    10
addr_state                0
dti                       0
delinq_2yrs               0
earliest_cr_line          0
inq_last_6mths            0
open_acc                  0
pub_rec                   0
revol_bal                 0
revol_util               50
total_acc                 0
last_credit_pull_d        2
pub_rec_bankruptcies    697
dtype: int64

In [59]:
#Here pub_rec_bankruptcies contains 697 null values i will remove it

In [60]:
credit.drop("pub_rec_bankruptcies",axis=1,inplace=True)

In [61]:
#now i will drop all the row which contain null values
credit = credit.dropna(axis=0)

In [62]:
credit.isnull().sum()

loan_amnt              0
term                   0
int_rate               0
installment            0
emp_length             0
home_ownership         0
annual_inc             0
verification_status    0
loan_status            0
purpose                0
title                  0
addr_state             0
dti                    0
delinq_2yrs            0
earliest_cr_line       0
inq_last_6mths         0
open_acc               0
pub_rec                0
revol_bal              0
revol_util             0
total_acc              0
last_credit_pull_d     0
dtype: int64

In [63]:
#now we have no null values now

In [64]:
credit.dtypes.value_counts()

float64    11
object      9
int64       1
int32       1
dtype: int64

In [65]:
#so there are 9 columns which contain object type we need to convert them

In [66]:
object_type = credit.select_dtypes(include=["object"])

In [67]:
object_type.columns

Index(['term', 'home_ownership', 'verification_status', 'purpose', 'title',
       'addr_state', 'earliest_cr_line', 'revol_util', 'last_credit_pull_d'],
      dtype='object')

In [68]:
#we can convert the revol_util to numeric,just remove %

In [69]:
credit['revol_util'] = credit['revol_util'].str.strip('%').astype('float')

In [70]:
#'earliest_cr_line' and 'last_credit_pull_d' contain date values which will require good amount of 
#feature enginnering to make it useful so lets drop it for now

In [71]:
credit.drop(['last_credit_pull_d','earliest_cr_line'],axis=1,inplace=True)

In [72]:
#now purpose and title columns are some kind of same
#we have to remove one of it
#i will keep purpose column since less amount of discrate values
#lets drop title column

In [73]:
credit.drop("title",axis=1,inplace=True)

In [74]:
credit

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc
0,5000.0,36 months,10.65,162.87,10,RENT,24000.00,Verified,1,credit_card,AZ,27.65,0.0,1.0,3.0,0.0,13648.0,83.70,9.0
1,2500.0,60 months,15.27,59.83,0,RENT,30000.00,Source Verified,0,car,GA,1.00,0.0,5.0,3.0,0.0,1687.0,9.40,4.0
2,2400.0,36 months,15.96,84.33,10,RENT,12252.00,Not Verified,1,small_business,IL,8.72,0.0,2.0,2.0,0.0,2956.0,98.50,10.0
3,10000.0,36 months,13.49,339.31,10,RENT,49200.00,Source Verified,1,other,CA,20.00,0.0,1.0,10.0,0.0,5598.0,21.00,37.0
5,5000.0,36 months,7.90,156.46,3,RENT,36000.00,Source Verified,1,wedding,AZ,11.20,0.0,3.0,9.0,0.0,7963.0,28.30,12.0
6,7000.0,60 months,15.96,170.08,8,RENT,47004.00,Not Verified,1,debt_consolidation,NC,23.51,0.0,1.0,7.0,0.0,17726.0,85.60,11.0
7,3000.0,36 months,18.64,109.43,9,RENT,48000.00,Source Verified,1,car,CA,5.35,0.0,2.0,4.0,0.0,8221.0,87.50,4.0
8,5600.0,60 months,21.28,152.39,4,OWN,40000.00,Source Verified,0,small_business,CA,5.55,0.0,2.0,11.0,0.0,5210.0,32.60,13.0
9,5375.0,60 months,12.69,121.45,0,RENT,15000.00,Verified,0,other,TX,18.08,0.0,0.0,2.0,0.0,9279.0,36.50,3.0
10,6500.0,60 months,14.65,153.45,5,OWN,72000.00,Not Verified,1,debt_consolidation,AZ,16.12,0.0,2.0,14.0,0.0,4032.0,20.60,23.0


In [75]:
#so finally from 52 columns we got to 19 columns

In [76]:
credit['term'].value_counts()

 36 months    29041
 60 months     9667
Name: term, dtype: int64

In [77]:
#lets strip months and convert it to int type

In [78]:
credit['term'] = credit['term'].str.strip('months').astype('int')

In [79]:
credit.head(2)

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc
0,5000.0,36,10.65,162.87,10,RENT,24000.0,Verified,1,credit_card,AZ,27.65,0.0,1.0,3.0,0.0,13648.0,83.7,9.0
1,2500.0,60,15.27,59.83,0,RENT,30000.0,Source Verified,0,car,GA,1.00,0.0,5.0,3.0,0.0,1687.0,9.4,4.0


In [80]:
#home_ownership,verification_status,purpose,addr_state lets encode them as dummy variables to work.

In [81]:
cat_columns=['home_ownership','verification_status','purpose','addr_state']

In [82]:
dummy_col = pd.get_dummies(credit[cat_columns])

In [83]:
credit = pd.concat([credit,dummy_col],axis=1)

In [84]:
credit.drop(cat_columns,axis=1,inplace=True)

In [85]:
credit.head(2)

,loan_amnt,term,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,...,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY
0,5000.0,36,10.65,162.87,10,24000.0,1,27.65,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,2500.0,60,15.27,59.83,0,30000.0,0,1.00,0.0,5.0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
y = credit['loan_status']

In [87]:
x = credit.drop('loan_status',axis=1)

In [88]:
#so here x is the independent variable and y is the dependent/targeted variable

In [89]:
#so in this type of situation we are not looking for accuracy...We will be looking for false +ve values

In [90]:
#False +ve means we are thinking he/she will give the money but actually he/she is not giving the money
#so it is loss for the investor
#so we have to lower the false +ve values and increase the true +ve values

In [91]:
#It is a binary classification problem lets first go for logistic regression
#before that lets split the data in training and testing set (80%,20%)
from sklearn.cross_validation import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [92]:
from sklearn.linear_model import LogisticRegression
obj = LogisticRegression()
obj.fit(x_train,y_train)
y_pred = obj.predict(x_test)

In [93]:
#lets find out the false +ve and true +ve values

In [94]:
(38708*20)/100

7741.6

In [95]:
#False positives
fp_filter = (y_pred == 1) & (y_test == 0)
fp = len(y_pred[fp_filter])

# True positives.
tp_filter = (y_pred == 1) & (y_test == 1)
tp = len(y_pred[tp_filter])

# False negatives.
fn_filter = (y_pred == 0) & (y_test == 1)
fn = len(y_pred[fn_filter])

# True negatives
tn_filter = (y_pred == 0) & (y_test == 0)
tn = len(y_pred[tn_filter])

In [96]:
print(fp,tp,fn,tn)

1138 6570 26 8


In [97]:
#so we need to calculate false +ve rate and true +ve rate

In [98]:
false_positive_rate = fp/(fp+tn)
true_positive_rate = tp/(tp+fn)

In [99]:
print(false_positive_rate,true_positive_rate)

0.9930191972076788 0.9960582171012735


In [100]:
#here we can see false positive rate is very high ....we have to decrease it since it will be loss for
#investor for such high false +ve rate 
#so we have to penalize the algorithm if it makes a wrong prediction in 0.

In [101]:
def all_values(y_pred,y_test):
    fp_filter = (y_pred == 1) & (y_test == 0)#false +ve
    fp = len(y_pred[fp_filter])
    tp_filter = (y_pred == 1) & (y_test == 1)#true +ve
    tp = len(y_pred[tp_filter])
    fn_filter = (y_pred == 0) & (y_test == 1)#false -ve
    fn = len(y_pred[fn_filter])
    tn_filter = (y_pred == 0) & (y_test == 0)#true -ve
    tn = len(y_pred[tn_filter])
    false_positive_rate = fp/(fp+tn)
    print("the value of false_positive_rate is",false_positive_rate)
    true_positive_rate = tp/(tp+fn)
    print("the value of true_positive_rate is",true_positive_rate)

In [102]:
obj = LogisticRegression(class_weight='balanced')
obj.fit(x_train,y_train)
y_pred = obj.predict(x_test)
all_values(y_pred,y_test)

the value of false_positive_rate is 0.3403141361256545
the value of true_positive_rate is 0.6479684657368102


In [103]:
#now it is better than previous model
#now we can make it more accurate by applying penalty by ourselves

In [104]:
penalty = {0:10,1:1}

In [105]:
obj = LogisticRegression(class_weight=penalty)
obj.fit(x_train,y_train)
y_pred = obj.predict(x_test)
all_values(y_pred,y_test)

the value of false_positive_rate is 0.08202443280977312
the value of true_positive_rate is 0.24848392965433597


In [106]:
#by giving this penalty of our own we are getting very good result 8% for false +ve and 24% for true positive

In [107]:
#Lets apply random-forest algorithm

In [108]:
from sklearn.ensemble import RandomForestClassifier
obj1 = RandomForestClassifier(class_weight="balanced")

In [109]:
obj1.fit(x_train,y_train)
y_pred = obj1.predict(x_test)
all_values(y_pred,y_test)

the value of false_positive_rate is 0.9467713787085514
the value of true_positive_rate is 0.9748332322619769


In [110]:
#it is not at all good because random forest is highly biased towards the most occuring values
#may be we can give more harsh penalty lets see

In [111]:
penalty1 = {0:300,1:1}
obj1 = RandomForestClassifier(class_weight=penalty1)
obj1.fit(x_train,y_train)
y_pred = obj1.predict(x_test)
all_values(y_pred,y_test)


the value of false_positive_rate is 0.9336823734729494
the value of true_positive_rate is 0.9784718010915706


In [112]:
#even if we are around 300 it is not affecting ...so random forest is not all a good choice

In [113]:
#so logistic regression gives the best result

In [114]:
#Lets apply anomaly detection algorithm an see that can it make better prediction than logistic regression

In [115]:
credit.head(4)

,loan_amnt,term,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,...,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY
0,5000.0,36,10.65,162.87,10,24000.0,1,27.65,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
1,2500.0,60,15.27,59.83,0,30000.0,0,1.00,0.0,5.0,...,0,0,0,0,0,0,0,0,0,0
2,2400.0,36,15.96,84.33,10,12252.0,1,8.72,0.0,2.0,...,0,0,0,0,0,0,0,0,0,0
3,10000.0,36,13.49,339.31,10,49200.0,1,20.00,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [116]:
credit['loan_status'].value_counts()

1    33093
0     5615
Name: loan_status, dtype: int64

In [117]:
credit_one = credit[credit['loan_status']==1]
x = credit_one.drop('loan_status',axis=1)
y = credit_one['loan_status']

In [118]:
x_train_one,x_test_one,y_train_one,y_test_one = train_test_split(x,y,test_size=0.2,random_state=0)

In [119]:
#so x_train_one only contains successful values


In [120]:
credit_one = credit[credit['loan_status']==0]
x_two = credit_one.drop('loan_status',axis=1)
y_two = credit_one['loan_status']

In [121]:
test_final = pd.concat([x_two,x_test_one])

In [122]:
len(test_final)#it contains both +ve and -ve values

12234

In [123]:
test_final_outcome = pd.concat([y_two,y_test_one])

In [124]:
x_train_one.head(3)

,loan_amnt,term,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,...,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY
34426,9000.0,36,11.48,296.72,10,110000.0,11.02,0.0,1.0,17.0,...,0,0,0,0,0,0,0,0,0,0
22750,5000.0,36,13.72,170.21,1,76000.0,9.24,1.0,1.0,5.0,...,0,0,0,0,0,0,0,0,0,0
19541,3000.0,36,7.29,93.03,8,30000.0,26.96,0.0,2.0,4.0,...,0,0,0,0,0,0,0,0,0,0


In [125]:
mu=[]
sigma=[]
def cal_mu(s):
    return np.mean(s)
mu = x_train_one.apply(cal_mu)

In [126]:
def cal_sigma(s):
    return np.std(s)
sigma = x_train_one.apply(cal_sigma)

In [127]:
len(sigma)

86

In [128]:
all_index = test_final.index

In [129]:
prob={}
product=1 
for i in range(0,len(test_final)): #for test rows
    for j in range(0,85): #for features
        product=product*(1/(sigma[j]*np.sqrt(2*np.pi)))*np.exp(-(test_final.iloc[i][j] - mu[j])**2 / (2 * sigma[j]**2)) 
    prob[i] = product    
    product=1        

In [130]:
e=0.05
anomaly=[]
for key,values in prob.items():
    if values<200:
        anomaly.append(key)
        

In [131]:
anomaly[3]

9

In [132]:
test_final.head(1)

,loan_amnt,term,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,...,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY
1,2500.0,60,15.27,59.83,0,30000.0,1.0,0.0,5.0,3.0,...,0,0,0,0,0,0,0,0,0,0


In [133]:
test_final['predicted_values']=5

In [134]:
test_final.head()

,loan_amnt,term,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,...,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,predicted_values
1,2500.0,60,15.27,59.83,0,30000.0,1.00,0.0,5.0,3.0,...,0,0,0,0,0,0,0,0,0,5
8,5600.0,60,21.28,152.39,4,40000.0,5.55,0.0,2.0,11.0,...,0,0,0,0,0,0,0,0,0,5
9,5375.0,60,12.69,121.45,0,15000.0,18.08,0.0,0.0,2.0,...,0,1,0,0,0,0,0,0,0,5
12,9000.0,36,13.49,305.38,0,30000.0,10.08,0.0,1.0,4.0,...,0,0,0,1,0,0,0,0,0,5
14,10000.0,36,10.65,325.74,3,100000.0,7.06,0.0,2.0,14.0,...,0,0,0,0,0,0,0,0,0,5


In [135]:
test_final.iloc[0,86]

5

In [136]:
for i in range(0,len(test_final)):
    if i in anomaly:
        test_final.iloc[i,86]=0
    else:
        test_final.iloc[i,86]=1

In [137]:
test_final.head()

,loan_amnt,term,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,...,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,predicted_values
1,2500.0,60,15.27,59.83,0,30000.0,1.00,0.0,5.0,3.0,...,0,0,0,0,0,0,0,0,0,0
8,5600.0,60,21.28,152.39,4,40000.0,5.55,0.0,2.0,11.0,...,0,0,0,0,0,0,0,0,0,0
9,5375.0,60,12.69,121.45,0,15000.0,18.08,0.0,0.0,2.0,...,0,1,0,0,0,0,0,0,0,1
12,9000.0,36,13.49,305.38,0,30000.0,10.08,0.0,1.0,4.0,...,0,0,0,1,0,0,0,0,0,1
14,10000.0,36,10.65,325.74,3,100000.0,7.06,0.0,2.0,14.0,...,0,0,0,0,0,0,0,0,0,1


In [138]:
all_values(test_final['predicted_values'],test_final_outcome)

the value of false_positive_rate is 0.5006233303650935
the value of true_positive_rate is 0.5293851034899532
